## Collaborative filtering

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/final_train_data.csv')
df.head()
#df = df[:100000]

/tmp/ipython-input-1027023361.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/final_train_data.csv')


,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento,age_bin,products
0,2015-01-28,1375586,0,ES,H,35.0,2015-01-12,0,6,1,...,KHL,0,1,29.0,MALAGA,1,87218.10,02 - PARTICULARES,30-39,['Current Accounts']
1,2015-01-28,1050611,0,ES,V,23.0,2012-08-10,0,35,1,...,KHE,0,1,13.0,CIUDAD REAL,0,35548.74,03 - UNIVERSITARIO,18-29,['Current Accounts']
2,2015-01-28,1050612,0,ES,V,23.0,2012-08-10,0,35,1,...,KHE,0,1,13.0,CIUDAD REAL,0,122179.11,03 - UNIVERSITARIO,18-29,['Current Accounts']
3,2015-01-28,1050613,0,ES,H,22.0,2012-08-10,0,35,1,...,KHD,0,1,50.0,ZARAGOZA,0,119775.54,03 - UNIVERSITARIO,18-29,['Short-term Deposits']
4,2015-01-28,1050614,0,ES,V,23.0,2012-08-10,0,35,1,...,KHE,0,1,50.0,ZARAGOZA,1,88474.38,03 - UNIVERSITARIO,18-29,['Current Accounts']


In [ ]:
df.shape

(10974687, 24)

In [ ]:
df.dtypes

,0
fecha_dato,object
ncodpers,int64
ind_empleado,int64
pais_residencia,object
sexo,object
age,float64
fecha_alta,object
ind_nuevo,int64
antiguedad,object
indrel,int64


In [ ]:
# converts date to datetime format and get month
df['fecha_dato'] = pd.to_datetime(df['fecha_dato'])
df['month'] = df['fecha_dato'].dt.to_period('M')

# converts products (strings) into list of values
def ensure_list(x):
    if isinstance(x, list):
        return x
    if pd.isna(x):
        return []
    s = str(x).strip()
    if s.startswith('[') and s.endswith(']'):
        s = s[1:-1]  # drop [ ]
    if s == '':
        return []
    # split on commas and clean quotes/spaces
    return [t.strip().strip("'").strip('"') for t in s.split(',')]

df['products'] = df['products'].apply(ensure_list)

In [ ]:
df.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento,age_bin,products,month
0,2015-01-28,1375586,0,ES,H,35.0,2015-01-12,0,6,1,...,0,1,29.0,MALAGA,1,87218.10,02 - PARTICULARES,30-39,[Current Accounts],2015-01
1,2015-01-28,1050611,0,ES,V,23.0,2012-08-10,0,35,1,...,0,1,13.0,CIUDAD REAL,0,35548.74,03 - UNIVERSITARIO,18-29,[Current Accounts],2015-01
2,2015-01-28,1050612,0,ES,V,23.0,2012-08-10,0,35,1,...,0,1,13.0,CIUDAD REAL,0,122179.11,03 - UNIVERSITARIO,18-29,[Current Accounts],2015-01
3,2015-01-28,1050613,0,ES,H,22.0,2012-08-10,0,35,1,...,0,1,50.0,ZARAGOZA,0,119775.54,03 - UNIVERSITARIO,18-29,[Short-term Deposits],2015-01
4,2015-01-28,1050614,0,ES,V,23.0,2012-08-10,0,35,1,...,0,1,50.0,ZARAGOZA,1,88474.38,03 - UNIVERSITARIO,18-29,[Current Accounts],2015-01


In [ ]:

# 1) Long format: one row per (user, month, product)

long_df = (
    df[['ncodpers','month','products']]
      .explode('products', ignore_index=True)
      .dropna(subset=['products'])
      .rename(columns={'products':'product'})
      .drop_duplicates(['ncodpers','month','product'])
)


In [ ]:
long_df.head()

,ncodpers,month,product
0,1375586,2015-01,Current Accounts
1,1050611,2015-01,Current Accounts
2,1050612,2015-01,Current Accounts
3,1050613,2015-01,Short-term Deposits
4,1050614,2015-01,Current Accounts


In [ ]:
long_df['product'] = long_df['product'].astype('category')
long_df['ncodpers'] = long_df['ncodpers'].astype('int64')

In [ ]:
# Time based split

target_month = long_df['month'].max()
prev_month   = target_month - 1

In [ ]:
print(target_month)
print(prev_month)

2016-05
2016-04


In [ ]:
# train set has userid and products set upto April 2016
# eval set has userid and products set only in April 2016
# target set has userid and products set in May 2016

train_long = long_df[long_df['month'] <= prev_month].copy()
prev_long  = long_df[long_df['month'] == prev_month][['ncodpers','product']]
tgt_long   = long_df[long_df['month'] == target_month][['ncodpers','product']]

# users to evaluate
users_eval = tgt_long['ncodpers'].unique()

# dict to map userid and products of prev and target set
prev_set_by_user = prev_long.groupby('ncodpers')['product'].apply(set).to_dict()
tgt_set_by_user  = tgt_long.groupby('ncodpers')['product'].apply(set).to_dict()

def actual_new_products(u):
    return tgt_set_by_user.get(u, set()) - prev_set_by_user.get(u, set())

In [ ]:
# Building train set user–item sparse matrix

pairs = train_long[['ncodpers','product']].drop_duplicates().copy()
pairs['product'] = pairs['product'].astype('category')

# setting users as rows
users_index = pd.Index(pairs['ncodpers'].unique(), name='user')

# setting products as columns
items_index = pairs['product'].cat.categories

user2row = pd.Series(np.arange(len(users_index), dtype=np.int32), index=users_index)
rows = pairs['ncodpers'].map(user2row).to_numpy()

cols = pairs['product'].cat.codes.to_numpy(dtype=np.int32)

# creating sparse matrix setting 1 for interactions
data = np.ones(len(pairs), dtype=np.float32)

n_users, n_items = len(users_index), len(items_index)
X = csr_matrix((data, (rows, cols)), shape=(n_users, n_items), dtype=np.float32)

# Popular items for fallback
global_top = (pairs['product'].value_counts().index.tolist())


example:  


ncodpers | Current Accounts | Short-term Deposits |
---------|--------------------|---------------------|
1375586  |         1          |          0          |
1050611  |         1          |          0          |
1050612  |         1          |          0          |
1050613  |         0          |          1          |
1050614  |         1          |          0          |

In [ ]:
print(n_users)
print(n_items)

720721
24


In [ ]:
print(global_top)

['Current Accounts', 'Direct Debit', 'Particular Account', 'Payroll Account', 'E-account', 'Pensions Account', 'Payroll', 'Credit Card', 'Long-term Deposits', 'Taxes', 'Particular Plus Account', 'Securities', 'Funds', 'Más particular Account', 'Pensions', 'Short-term Deposits', 'Mortgage', 'Home Account', 'Loans', 'Junior Account', 'Medium-term Deposits', 'Derivada Account', 'Saving Account', 'Guarantees']


In [ ]:

# Fit User-based KNN using cosine similarity

K = 50
nbrs = NearestNeighbors(metric='cosine', algorithm='brute',
                        n_neighbors=min(K+1, max(2, n_users)))
nbrs.fit(X)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=3)

In [ ]:

# top 7 recommendations

item_lookup = np.array(items_index)
item2col = {p:i for i,p in enumerate(items_index)}

def recommend_top_n_for_user(u, n=7):
    # get products the user already owned in the previous month
    prev_owned = prev_set_by_user.get(u, set())

    # if user not in training data, assign global products
    if u not in user2row.index:
        recs = [p for p in global_top if p not in prev_owned][:n]
        return recs

    uidx = user2row.loc[u]
    urow = X[uidx]

    # using nearest neighbors
    distances, indices = nbrs.kneighbors(urow, n_neighbors=min(K+1, n_users))
    distances = distances.ravel()
    indices   = indices.ravel()

    # convert distance to similarity
    sims = 1.0 - distances

    indices = indices[1:]
    sims    = sims[1:]

    if indices.size == 0 or np.all(sims <= 0):
        return [p for p in global_top if p not in prev_owned][:n]

    # Weighted neighbor sum
    neighbors_matrix = X[indices]
    scores = np.asarray(neighbors_matrix.multiply(sims[:, None]).sum(axis=0)).ravel()

    # exclude already owned at prev_month
    for p in prev_owned:
        j = item2col.get(p, None)
        if j is not None:
            scores[j] = -np.inf

    if not np.isfinite(scores).any():
        return [p for p in global_top if p not in prev_owned][:n]

    # take top-n
    top_idx = np.argpartition(-scores, kth=min(n-1, len(scores)-1))[:n]
    top_idx = top_idx[np.argsort(-scores[top_idx])]
    preds = item_lookup[top_idx].tolist()

    if len(preds) < n:
        tail = [p for p in global_top if (p not in prev_owned) and (p not in preds)]
        preds += tail[:(n - len(preds))]
    return preds[:n]

In [ ]:

# MAP@7 metrics

def average_precision_at_k(true_set, pred_list, k=7):
    if not true_set:
        return 0.0
    hits = 0
    ap = 0.0
    for i, p in enumerate(pred_list[:k], start=1):
        if p in true_set:
            hits += 1
            ap += hits / i
    return ap / min(len(true_set), k)

def map7(user_ids):
    vals = []
    for u in user_ids:
        true_new = actual_new_products(u)
        pred7 = recommend_top_n_for_user(u, n=7)
        vals.append(average_precision_at_k(true_new, pred7, k=7))
    return float(np.mean(vals)) if vals else 0.0

In [ ]:
"""

# 7) Evaluate

score = map7(users_eval)
print(f"MAP@7 on {str(target_month)}: {score:.6f}")

# Inspect a few
for u in list(users_eval)[:5]:
    print("\nUser:", u)
    print("True new:", sorted(actual_new_products(u)))
    print("Pred@7 :", recommend_top_n_for_user(u, 7))
"""

' \n# -----------------------------\n# 7) Evaluate\n# -----------------------------\nscore = map7(users_eval)\nprint(f"MAP@7 on {str(target_month)}: {score:.6f}")\n\n# Inspect a few\nfor u in list(users_eval)[:5]:\n    print("\nUser:", u)\n    print("True new:", sorted(actual_new_products(u)))\n    print("Pred@7 :", recommend_top_n_for_user(u, 7))\n'

In [ ]:
# running on 100k sample
N = 100000
users_sample = pd.Series(users_eval).sample(n=min(N, len(users_eval)), random_state=42).to_numpy()
score = map7(users_sample)
print(f"MAP@7 on random sample ({len(users_sample)}): {score:.6f}")


MAP@7 on random sample (100000): 0.030390
